# 텐서플로우를 이용한 GANs (Generative Adversarial Networks) 예제

GANs는 오토인코더와 마찬가지로 대표적인 unsupervised model의 하나입니다. 
* 위폐범은 위조화폐를 정교하게 만들고자 최선을 다합니다. 즉 경찰이 실수할 확률을 높이기 위해 노력합니다.
* 경찰은 위폐를 감별하기 위해 최선을 다합니다. 즉 자신이 실수할 확률을 낮추기 위해 노력합니다. 
* 위폐범(G)의 네트워크를 Generative Network라고 합니다. 
* 경찰(D)의 네트워크를 Discriminator Network라고 합니다. 
* G는 D의 실수할 확률을 높이기 위해 노력하고 (maximize), D는 자신의 실수 확률을 낮추기 위해 노력하므로 (minimize) 이는 minimax problem입니다.
* GANs에서는 어려운 확률분포를 다루는 대신에 확률로부터 생성된 샘플을 다룹니다.
* G는 Z를 입력으로 받기 때문에 G(Z)로 표현하는데, 이때 Z가 바로 확률분포와 맵핑되는 prior라는 개념입니다. Random noise가 됩니다.
* 이때 G(Z)의 결과물은 물론 위폐가 됩니다.
* D는 이미지 X를 입력으로 받기 때문에 D(X)로 표현하며, D(X)의 결과물은 확률 (0~1) 이 됩니다.
* 양자의 균형을 맞추는 평형 상태에 이르면 G는 진짜 화폐와 100% 동일한 화폐를 만들게 되며, D가 이를 감별할 확률은 0.5가 되게 됩니다.

### 1. 필요한 모듈들을 불러 옵니다.

In [32]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import os

from tensorflow.examples.tutorials.mnist import input_data

In [33]:
mnist = input_data.read_data_sets('data/mnist', one_hot=True)

Extracting data/mnist/train-images-idx3-ubyte.gz
Extracting data/mnist/train-labels-idx1-ubyte.gz
Extracting data/mnist/t10k-images-idx3-ubyte.gz
Extracting data/mnist/t10k-labels-idx1-ubyte.gz


가중치의 초기화에 좋은 성능을 보이는 xavier initialization을 함수로 만들어서 이용합니다.

In [28]:
def xavier_init(size):
    in_dim = size[0]
    xavier_stddev = 1. / tf.sqrt(in_dim / 2.)
    return tf.random_normal(shape=size, stddev=xavier_stddev)

Discriminator 네트워크를 구축합니다.
* D(X) 를 통해 확률을 출력하게 됩니다.
* 노드수 : 784 --> 128 --> 1

In [39]:
# Discriminator Net

X = tf.placeholder(tf.float32, shape=[None, 784], name='X')

D_W1 = tf.Variable(xavier_init([784, 128]), name='D_W1')
# D_W1 = tf.get_variable('D_W1', [784,256], initializer=tf.contrib.layers.xavier_initializer()) 
D_b1 = tf.Variable(tf.zeros(shape=[128]), name='D_b1')

D_W2 = tf.Variable(xavier_init([128, 1]), name='D_W2')
# D_W2 = tf.get_variable('D_W2', [128,1], initializer=tf.contrib.layers.xavier_initializer())
D_b2 = tf.Variable(tf.zeros(shape=[1]), name='D_b2')

theta_D = [D_W1, D_W2, D_b1, D_b2]

Generator 네트워크를 구축합니다. 
* G(Z)를 통해 이미지를 출력하게 됩니다.
* 노드 수 : 100 --> 128 --> 784

In [ ]:
# Generator Net
Z = tf.placeholder(tf.float32, shape=[None, 100], name='Z')

G_W1 = tf.Variable(xavier_init([100, 128]), name='G_W1')
G_b1 = tf.Variable(tf.zeros(shape=[128]), name='G_b1')

G_W2 = tf.Variable(xavier_init([128, 784]), name='G_W2')
G_b2 = tf.Variable(tf.zeros(shape=[784]), name='G_b2')

theta_G = [G_W1, G_W2, G_b1, G_b2]

In [18]:
def sample_Z(m, n):
    return np.random.uniform(-1., 1., size=[m, n])

In [19]:
def generator(z):
    G_h1 = tf.nn.relu(tf.matmul(z, G_W1) + G_b1)
    G_log_prob = tf.matmul(G_h1, G_W2) + G_b2
    G_prob = tf.nn.sigmoid(G_log_prob)

    return G_prob

In [20]:
def discriminator(x):
    D_h1 = tf.nn.relu(tf.matmul(x, D_W1) + D_b1)
    D_logit = tf.matmul(D_h1, D_W2) + D_b2
    D_prob = tf.nn.sigmoid(D_logit)

    return D_prob, D_logit

In [21]:
def plot(samples):
    fig = plt.figure(figsize=(4, 4))
    gs = gridspec.GridSpec(4, 4)
    gs.update(wspace=0.05, hspace=0.05)

    for i, sample in enumerate(samples):
        ax = plt.subplot(gs[i])
        plt.axis('off')
        ax.set_xticklabels([])
        ax.set_yticklabels([])
        ax.set_aspect('equal')
        plt.imshow(sample.reshape(28, 28), cmap='Greys_r')

    return fig

GAN을 학습시키기 위한 Adversarial Process를 선언합니다.

In [22]:
G_sample = generator(Z)
D_real, D_logit_real = discriminator(X)
D_fake, D_logit_fake = discriminator(G_sample)

In [23]:
D_loss = -tf.reduce_mean(tf.log(D_real) + tf.log(1. - D_fake))
G_loss = -tf.reduce_mean(tf.log(D_fake))

In [ ]:
# Alternative losses:
# -------------------
# D_loss_real = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=D_logit_real, labels=tf.ones_like(D_logit_real)))
# D_loss_fake = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=D_logit_fake, labels=tf.zeros_like(D_logit_fake)))
# D_loss = D_loss_real + D_loss_fake
# G_loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=D_logit_fake, labels=tf.ones_like(D_logit_fake)))

In [ ]:
D_solver = tf.train.AdamOptimizer().minimize(D_loss, var_list=theta_D)
G_solver = tf.train.AdamOptimizer().minimize(G_loss, var_list=theta_G)

In [29]:
mb_size = 128
Z_dim = 100

In [30]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

if not os.path.exists('out/'):
    os.makedirs('out/')

i = 0

In [31]:
for it in range(100000):
    if it % 1000 == 0:
        samples = sess.run(G_sample, feed_dict={Z: sample_Z(16, Z_dim)})

        fig = plot(samples)
        plt.savefig('out/{}.png'.format(str(i).zfill(3)), bbox_inches='tight')
        i += 1
        plt.close(fig)

    X_mb, _ = mnist.train.next_batch(mb_size)

    _, D_loss_curr = sess.run([D_solver, D_loss], feed_dict={X: X_mb, Z: sample_Z(mb_size, Z_dim)})
    _, G_loss_curr = sess.run([G_solver, G_loss], feed_dict={Z: sample_Z(mb_size, Z_dim)})

    if it % 1000 == 0:
        print('Iter: {}'.format(it))
        print('D loss: {:.4}'. format(D_loss_curr))
        print('G_loss: {:.4}'.format(G_loss_curr))
        print()

Iter: 0
D loss: 1.475
G_loss: 2.61
()
Iter: 1000
D loss: 0.002581
G_loss: 8.531
()
Iter: 2000
D loss: 0.01967
G_loss: 6.47
()
Iter: 3000
D loss: 0.04834
G_loss: 5.831
()
Iter: 4000
D loss: 0.108
G_loss: 4.732
()
Iter: 5000
D loss: 0.1607
G_loss: 5.041
()
Iter: 6000
D loss: 0.434
G_loss: 4.554
()
Iter: 7000
D loss: 0.5812
G_loss: 3.518
()
Iter: 8000
D loss: 0.37
G_loss: 3.917
()
Iter: 9000
D loss: 0.5032
G_loss: 2.94
()
Iter: 10000
D loss: 0.3547
G_loss: 2.939
()
Iter: 11000
D loss: 0.3589
G_loss: 3.141
()
Iter: 12000
D loss: 0.4143
G_loss: 2.984
()
Iter: 13000
D loss: 0.8226
G_loss: 2.595
()
Iter: 14000
D loss: 0.6443
G_loss: 2.319
()
Iter: 15000
D loss: 0.548
G_loss: 2.131
()
Iter: 16000
D loss: 0.5133
G_loss: 2.252
()
Iter: 17000
D loss: 0.5358
G_loss: 2.367
()
Iter: 18000
D loss: 0.6386
G_loss: 2.082
()
Iter: 19000
D loss: 0.5908
G_loss: 2.251
()
Iter: 20000
D loss: 0.6895
G_loss: 2.39
()
Iter: 21000
D loss: 0.6093
G_loss: 2.084
()
Iter: 22000
D loss: 0.6968
G_loss: 2.367
()
Iter: 2

KeyboardInterrupt: 